In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50, EfficientNetB0, VGG16
from tensorflow.keras.models import Model
import os

In [2]:
# Directory structure
TRAIN_DIR = "PROCESSED_DATA/TRAINING_DATA/TRAINING_AUGMENTED_DATA"
VALID_DIR = "PROCESSED_DATA/VALIDATION_DATA/"
TEST_DIR = "PROCESSED_DATA/TEST_DATA/"

# Image Parameters
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Load Data
datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical')

val_generator = datagen.flow_from_directory(
    VALID_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical')

test_generator = datagen.flow_from_directory(
    TEST_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False)


Found 6642 images belonging to 11 classes.
Found 440 images belonging to 11 classes.
Found 220 images belonging to 11 classes.


In [3]:
# Build a CNN model
def build_cnn():
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(224, 224, 3)),
        MaxPooling2D(2,2),
        Conv2D(64, (3,3), activation='relu', padding='same'),
        MaxPooling2D(2,2),
        Conv2D(128, (3,3), activation='relu', padding='same'),
        MaxPooling2D(2,2),
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(train_generator.num_classes, activation='softmax')
    ])
    return model

# Build a Transfer Learning Model
def build_transfer_learning(model_name):
    if model_name == 'ResNet50':
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    elif model_name == 'EfficientNet':
        base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    elif model_name == 'VGG16':
        base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    else:
        raise ValueError("Invalid model name. Choose from 'ResNet50', 'EfficientNet', or 'VGG16'")
    
    base_model.trainable = False  # Freeze the base model layers
    
    x = Flatten()(base_model.output)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    output_layer = Dense(train_generator.num_classes, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=output_layer)
    return model


#### Standard CNN without transfer learning

In [4]:
# Select the model to train
use_transfer_learning = False  # Change to False for a standard CNN
transfer_model_name = 'ResNet50'  # Options: 'ResNet50', 'EfficientNet', 'VGG16'

if use_transfer_learning:
    model = build_transfer_learning(transfer_model_name)
else:
    model = build_cnn()

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10
)


Epoch 1/10
208/208 [==============================] - 208s 994ms/step - loss: 2.3308 - accuracy: 0.1593 - val_loss: 2.1978 - val_accuracy: 0.2227
Epoch 2/10
208/208 [==============================] - 214s 1s/step - loss: 2.1478 - accuracy: 0.2543 - val_loss: 2.1072 - val_accuracy: 0.3227
Epoch 3/10
208/208 [==============================] - 226s 1s/step - loss: 2.0000 - accuracy: 0.3129 - val_loss: 2.0016 - val_accuracy: 0.3295
Epoch 4/10
208/208 [==============================] - 228s 1s/step - loss: 1.8567 - accuracy: 0.3683 - val_loss: 1.9427 - val_accuracy: 0.3477
Epoch 5/10
208/208 [==============================] - 229s 1s/step - loss: 1.6967 - accuracy: 0.4262 - val_loss: 1.8842 - val_accuracy: 0.4045
Epoch 6/10
208/208 [==============================] - 228s 1s/step - loss: 1.5902 - accuracy: 0.4642 - val_loss: 1.9006 - val_accuracy: 0.3955
Epoch 7/10
208/208 [==============================] - 230s 1s/step - loss: 1.4560 - accuracy: 0.5161 - val_loss: 1.7859 - val_accuracy: 0.4

In [5]:
# Evaluate on test data
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc:.4f}")

# Save the model
model.save("pest_classifier_cnn.h5")


7/7 [==============================] - 3s 370ms/step - loss: 1.9832 - accuracy: 0.3682
Test Accuracy: 0.3682


#### Use transfer leaning: EfficientNet

In [ ]:
# Select the model to train
use_transfer_learning = True  # Change to False for a standard CNN
transfer_model_name = 'EfficientNet'  # Options: 'ResNet50', 'EfficientNet', 'VGG16'

if use_transfer_learning:
    model = build_transfer_learning(transfer_model_name)
else:
    model = build_cnn()

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10
)


In [ ]:
# Evaluate on test data
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc:.4f}")

# Save the model
model.save("pest_classifier_cnn_efficientnet.h5")
